# Purpose and set up
Created by Emily Jane Dennis in August 2020, compiling from scripts from Zahra and Austin.

The purpose of this notebook is to walk a user through the steps to take a published atlas and transform its information to apply to our Princeton Rat Atlas, and so it can be visualized in Neuroglancer. 

First, you should run through all the set up instructions in the rat_BrainPipe README

Next you need to find the locations of 
1. median_image.tif which is, for now, our "atlas"
2. rat_BrainPipe repo
3. your new atlas BRAIN
4. your new atlas ANNOTATION file

You first need to use elastix to get your new atlas brain into PRA space. This is accomplished by running the code below. Note this only needs to happen once.

**Make sure you have changed the paths in directorydeterminer**

In [26]:
import os, sys, time, shutil
import tifffile as tif
from xvfbwrapper import Xvfb
sys.path.append("/home/emilyjanedennis/Desktop/GitHub/rat_BrainPipe/")
from scipy.ndimage.interpolation import zoom
from tools.utils.io import makedir
from tools.registration.register import change_interpolation_order
from tools.registration.register import transformix_command_line_call
from tools.registration.register import elastix_wrapper
from tools.registration.transform_list_of_points import modify_transform_files
from tools.utils.directorydeterminer import directorydeterminer
from tools.imageprocessing import preprocessing


In [35]:
vdisplay = Xvfb()
vdisplay.start()

systemdirectory="/home/emilyjanedennis/"

In [43]:
inputdictionary = {
    os.path.join(systemdirectory, "Desktop/w122_43p_low_thres/full_sizedatafld/_ch00"):
    [["regch", "00"]],
    os.path.join(systemdirectory, "Desktop/w122_43p_low_thres/full_sizedatafld/_ch00"):
    [["injch", "00"]]
}


In [44]:

# Required inputs
params = {
    "systemdirectory":  systemdirectory,  # don"t need to touch
    "inputdictionary": inputdictionary,  # don"t need to touch
    "outputdirectory": os.path.join(systemdirectory, "Desktop/w122"),
    # (5.0,5.0,3), #micron/pixel: 5.0um/pix for 1.3x; 1.63um/pix for 4x
    "xyz_scale": (5, 5, 10),
    "tiling_overlap": 43.0,  # percent overlap taken during tiling
    "stitchingmethod": "blending",  # "terastitcher" or "blending"
    "AtlasFile": os.path.join(systemdirectory, "Desktop/for_registration_to_lightsheet/WHS_SD_rat_T2star_v1.01_atlas.tif"),
    # path to annotation file for structures
    "annotationfile": os.path.join(systemdirectory, "Desktop/for_registration_to_lightsheet/WHS_SD_rat_atlas_v3_annotation.tif"),
    "blendtype": "sigmoidal",  # False/None, "linear", or "sigmoidal"
    # blending between tiles, usually sigmoidal;
    # False or None for images where blending would be detrimental
    # True = calculate mean intensity of overlap between tiles shift higher
    # of two towards lower - useful for images where relative intensity
    # is not important (i.e. tracing=True, cFOS=False)
    "intensitycorrection": True,
    "resizefactor": 3,  # in x and y #normally set to 5 for 4x objective,
    # 3 for 1.3x obj
    "rawdata": True,  # set to true if raw data is taken from scope and
    # images need to be flattened; functionality for
    # rawdata =False has not been tested**
    # Used to account for different orientation between brain and atlas.
    # Assumes XYZ ("0","1","2) orientation.
    # Pass strings NOT ints. "-0" = reverse the order of the xaxis.
    # For better description see docstring from
    # tools.imageprocessing.orientation import fix_orientation;
    # ("2","1","0") for horizontal to sagittal,
    # Order of operations is reversing of axes BEFORE swapping axes.
    "finalorientation":  ("2", "1", "0"),
    "slurmjobfactor": 50
    # number of array iterations per arrayjob
    # since max job array on SPOCK is 1000
}
print(params)

{'systemdirectory': '/home/emilyjanedennis/', 'inputdictionary': {'/home/emilyjanedennis/Desktop/w122_43p_low_thres/full_sizedatafld/_ch00': [['injch', '00']]}, 'outputdirectory': '/home/emilyjanedennis/Desktop/w122', 'xyz_scale': (5, 5, 10), 'tiling_overlap': 43.0, 'stitchingmethod': 'blending', 'AtlasFile': '/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/WHS_SD_rat_T2star_v1.01_atlas.tif', 'annotationfile': '/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/WHS_SD_rat_atlas_v3_annotation.tif', 'blendtype': 'sigmoidal', 'intensitycorrection': True, 'resizefactor': 3, 'rawdata': True, 'finalorientation': ('2', '1', '0'), 'slurmjobfactor': 50}


'/home/emilyjanedennis/Desktop/GitHub/rat_BrainPipe/tools/analysis'

In [49]:
preprocessing.generateparamdict(os.getcwd(), **params)

IndexError: list index out of range

In [ ]:


        # preprocessing.updateparams("/", svnm = "param_dict_local.p",**params)
        # make a local copy
        if not os.path.exists(os.path.join(params["outputdirectory"],
                                           "lightsheet")):
            shutil.copytree(os.getcwd(), os.path.join(
                params["outputdirectory"],
                "lightsheet"),
                ignore=shutil.ignore_patterns(*(
                    ".pyc", "CVS",
                    ".git", "tmp", ".svn")))
            # copy run folder into output to save run info


In [38]:
elastix_wrapper(0, cores=8, **params)  # run elastix
vdisplay.stop()

FileNotFoundError: [Errno 2] No such file or directory: '/home/emilyjanedennis/Desktop/w122/param_dict.p'

'emilyjanedennis-System-Product-Name'

I set paths here on 20200808 to work on my local machine - EJD

In [21]:
# setting paths
src = "/home/emilyjanedennis/LightSheetData/brodyatlas/atlas/for_registration_to_lightsheet/"
ann = os.path.join(src, "WHS_SD_rat_atlas_v3_annotation.tif")
fx = "/home/emilyjanedennis/LightSheetData/brodyatlas/atlas/2019_meta_atlas/median_image.tif"
print(ann)

/home/emilyjanedennis/LightSheetData/brodyatlas/atlas/for_registration_to_lightsheet/WHS_SD_rat_atlas_v3_annotation.tif


In [22]:
# need to make MRI annotation larger (~140% of atlas?) to transform to PRA
watl = tif.imread(ann)
pra = tif.imread(fx)
zf,yf,xf = (pra.shape[0]/watl.shape[0])*1.4, (pra.shape[1]/watl.shape[1])*1.4, (pra.shape[2]/watl.shape[2])*1.4
print("\nzooming...")
watl_for_pra = zoom(watl, (zf,yf,xf), order = 1)


zooming...


In [ ]:
# saved out annotation volume
print("\nsaving zoomed volume...")
tif.imsave(os.path.join(src, "WHS_SD_rat_atlas_v3_annotation_for_pra_reg.tif"),
           watl_for_pra.astype("uint16"))

reg = os.path.join(src, "waxholm_to_pra")
a2r = [os.path.join(reg, xx) for xx in os.listdir(reg) if "Transform" in xx]; a2r.sort()

dst = os.path.join(src, "transformed_annotation_volume")
makedir(dst)

In [ ]:
# transformix
transformfiles = modify_transform_files(transformfiles=a2r, dst=dst)
[change_interpolation_order(xx,0) for xx in transformfiles]

In [ ]:
# change the parameter in the transform files that outputs 16bit images instead
for fl in transformfiles:# Read in the file
    with open(fl, "r") as file:
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace('(ResultImagePixelType "float")', '(ResultImagePixelType "short")')
    # Write the file out again
    with open(fl, "w") as file:
      file.write(filedata)

In [ ]:
# run transformix  
transformix_command_line_call(os.path.join(src, "WHS_SD_rat_atlas_v3_annotation_for_pra_reg.tif"), 
                              dst, transformfiles[-1])  